In [5]:
import pandas as pd

In [8]:
data_s1 = pd.read_csv('data/Stage_1.csv');
data_s2 = pd.read_csv('data/Stage_1.csv');

In [ ]:
def Stage_3(out_11,out_12,out_13,out_14,out_2,ox,T):
    out_3 = 71.05(1-exp(-0.1657*ox))(1-exp(-0.2*(T-100)))
    out_4 = 1.04 * out_3/100 * ( (out_14 + out_2/100 * (out_11+out_12+out_13)) )
    return out_4